In [ ]:
#To do:
#-Variable date range in single_HYCOM_SST_image's filter
#-Convert point to image and blend it with image of SST 
#  which needs Properties and Reducer
#-Apply filterBounds to collection i.e. the roi

# Notes
#-Opaque layer shows up based upon the order in which it was added to the map.

In [1]:
# Imports
import ee
import geemap
import geemap.colormaps as cm
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
import numpy as np

In [2]:
# Custom import from a different folder
import sys
sys.path.append('../project')
import project_functions

In [3]:
ee.Authenticate()

True

In [4]:
m = geemap.Map()

In [5]:
# ROI
roi = m.user_roi
if roi is None:
    roi = ee.Geometry.BBox(-76.0, 34.25, -73.0, 37.5)
    #roi = ee.Geometry.BBox(-77.0, 30.0, -60.0, 45.0)
    #m.add_layer(roi)
    m.center_object(roi)
    

In [6]:
# Get a single animal track as a pandas DataFrame

# Specify fullpath to input csv file containing multiple animal tracks
# i.e. the csv downloaded from movebank.org and load it into a pandas DataFrame
track_csv_fullpath = '../../project_data/track_data/Short_finned_pilot_whales_CRC_NW_Atlantic.csv'

# Let's look at just one of the tracks. From data exploration, I found GmTag142 to be 
# interesting i.e. it went through my region of interest
tag_id = 'GmTag142'  # e.g. 'GmTag142' or 'GmTag137'

# Call function to get the individual track, as pandas dataframe
myTrack = project_functions.get_movebank_track_from_csv(track_csv_fullpath, tag_id)

# Subset myTrack in time
myTrack_in_time_window = myTrack['timestamp_datetime']<datetime(2016,1,6)   # boolean
myTrack_subset = myTrack[myTrack_in_time_window]

# Reset index
myTrack_subset.reset_index()

    event-id                timestamp  location-long  location-lat  \
0  369777537  2014-06-11 17:49:22.000        -74.763        35.936   
1  369777538  2014-06-11 19:23:15.000        -74.810        35.891   
2  369777539  2014-06-11 20:26:13.000        -74.983        35.878   
3  369777540  2014-06-11 21:58:56.000        -74.829        35.920   
4  369777543  2014-06-12 00:03:31.000        -74.859        35.935   

  individual-local-identifier  timestamp_datetime  
0                    GmTag092 2014-06-11 17:49:22  
1                    GmTag092 2014-06-11 19:23:15  
2                    GmTag092 2014-06-11 20:26:13  
3                    GmTag092 2014-06-11 21:58:56  
4                    GmTag092 2014-06-12 00:03:31  


,index,event-id,timestamp,location-long,location-lat,individual-local-identifier,timestamp_datetime
0,4466,1033855502,2015-10-21 14:26:58.000,-74.77900,35.62500,GmTag142,2015-10-21 14:26:58
1,4467,1033855503,2015-10-21 15:23:06.000,-74.77200,35.62200,GmTag142,2015-10-21 15:23:06
2,4468,1033855505,2015-10-21 17:39:48.000,-74.75700,35.62900,GmTag142,2015-10-21 17:39:48
3,4469,1033855506,2015-10-21 21:20:51.000,-74.77600,35.62900,GmTag142,2015-10-21 21:20:51
4,4470,1033855508,2015-10-21 21:57:22.000,-74.77400,35.62500,GmTag142,2015-10-21 21:57:22
...,...,...,...,...,...,...,...
508,4974,1125787087,2016-01-05 10:52:11.000,-73.58870,36.10448,GmTag142,2016-01-05 10:52:11
509,4975,1125787088,2016-01-05 11:33:59.000,-73.65079,36.11283,GmTag142,2016-01-05 11:33:59
510,4976,1125787089,2016-01-05 14:09:08.000,-73.57721,36.22884,GmTag142,2016-01-05 14:09:08
511,4977,1125787092,2016-01-05 22:12:45.000,-73.23096,36.47164,GmTag142,2016-01-05 22:12:45


In [7]:
# Convert track from pandas DataFrame to geopandas GeoDataFrame
# First, specify geometry
geometry_myTrack = gpd.points_from_xy(myTrack_subset['location-long'],myTrack_subset['location-lat'])
gdf_myTrack = gpd.GeoDataFrame(myTrack_subset,geometry=geometry_myTrack,crs="EPSG:4326")

In [8]:
# Reset index
gdf_myTrack.reset_index()
gdf_myTrack

,event-id,timestamp,location-long,location-lat,individual-local-identifier,timestamp_datetime,geometry
4466,1033855502,2015-10-21 14:26:58.000,-74.77900,35.62500,GmTag142,2015-10-21 14:26:58,POINT (-74.779 35.625)
4467,1033855503,2015-10-21 15:23:06.000,-74.77200,35.62200,GmTag142,2015-10-21 15:23:06,POINT (-74.772 35.622)
4468,1033855505,2015-10-21 17:39:48.000,-74.75700,35.62900,GmTag142,2015-10-21 17:39:48,POINT (-74.757 35.629)
4469,1033855506,2015-10-21 21:20:51.000,-74.77600,35.62900,GmTag142,2015-10-21 21:20:51,POINT (-74.776 35.629)
4470,1033855508,2015-10-21 21:57:22.000,-74.77400,35.62500,GmTag142,2015-10-21 21:57:22,POINT (-74.774 35.625)
...,...,...,...,...,...,...,...
4974,1125787087,2016-01-05 10:52:11.000,-73.58870,36.10448,GmTag142,2016-01-05 10:52:11,POINT (-73.5887 36.10448)
4975,1125787088,2016-01-05 11:33:59.000,-73.65079,36.11283,GmTag142,2016-01-05 11:33:59,POINT (-73.65079 36.11283)
4976,1125787089,2016-01-05 14:09:08.000,-73.57721,36.22884,GmTag142,2016-01-05 14:09:08,POINT (-73.57721 36.22884)
4977,1125787092,2016-01-05 22:12:45.000,-73.23096,36.47164,GmTag142,2016-01-05 22:12:45,POINT (-73.23096 36.47164)


In [9]:
# Convert geopandas track to GEE feature class and add it as a layer to the map
fc_my_Track = geemap.gdf_to_ee(gdf_myTrack)
m.add_layer(fc_my_Track, {}, "Track")

In [10]:
fc_my_Track

In [11]:
geometry_myTrack


<GeometryArray>
[<POINT (-74.779 35.625)>, <POINT (-74.772 35.622)>, <POINT (-74.757 35.629)>,
 <POINT (-74.776 35.629)>, <POINT (-74.774 35.625)>, <POINT (-74.729 35.646)>,
 <POINT (-74.739 35.668)>,  <POINT (-74.77 35.685)>, <POINT (-74.774 35.691)>,
 <POINT (-74.778 35.698)>,
 ...
 <POINT (-74.847 35.665)>, <POINT (-73.698 36.154)>, <POINT (-73.681 36.166)>,
 <POINT (-73.656 36.175)>,  <POINT (-73.613 36.12)>, <POINT (-73.589 36.104)>,
 <POINT (-73.651 36.113)>, <POINT (-73.577 36.229)>, <POINT (-73.231 36.472)>,
 <POINT (-73.216 36.482)>]
Length: 513, dtype: geometry

In [12]:
# Extract array and convert to numpy 
df_timestamp_datetime = myTrack_subset['timestamp_datetime']
timestamp_datetime_array = np.array(df_timestamp_datetime)

In [13]:
# A function that returns a single HYCOM image, given a date
# For now, fudge variable date range
def single_HYCOM_SST_image(this_timestamp_datetime):
    # HYCOM SST collection
    collection = (ee.ImageCollection('HYCOM/sea_temp_salinity')
        .filter(ee.Filter.date("2019-03-06","2019-03-07"))
        .select('water_temp_0')
    )
    # return first image from the collection, scaled and offset to degrees C
    first_image = collection.first()
    #return first_image.multiply(0.001).add(20.0)
    # For some reason, the caller has trouble with max<min when I include the scaling and offset here;
    # therefore, let the caller apply the scaling and offset in the vis_params
    return first_image

In [14]:
# Initialize
image_list = []
# Loop on samples along the track--just a few for now until I get it working
#for frame_num in range(len(timestamp_datetime_array)):
for frame_num in range(4):    
    # Convert this point (along the track) into an image
    this_point = geometry_myTrack[frame_num]
    # Get the associated HYCOM SST image for this sample time
    this_timestamp_datetime = timestamp_datetime_array[frame_num].item()
    this_image = single_HYCOM_SST_image(this_timestamp_datetime)
    # Append this image to the image list
    image_list.append(this_image)

In [15]:
# Create an image collection from the image_list
custom_collection = ee.ImageCollection.fromImages(image_list)

min_val = (4-20)*1000
max_val = (27-20)*1000
vis_params_sst = {
  "min": min_val,
  "max": max_val,
  "palette": ['000000', '005aff', '43c8c8', 'fff700', 'ff0000'],
}

# Add slider
m.add_time_slider(custom_collection, vis_params_sst, time_interval=1)
m

Map(center=[35.875, -74.5], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright…

In [ ]:
# Test function before putting it in the loop--no longer needed
this_image = single_HYCOM_SST_image(this_timestamp_datetime)

min_val = 4
max_val = 27
vis_params_sst = {
  "min": min_val,
  "max": max_val,
  "palette": ['000000', '005aff', '43c8c8', 'fff700', 'ff0000'],
}

#m.add_layer(this_image, vis_params, "single_SST")
#m